In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
import cv2
from scipy.ndimage.filters import gaussian_filter

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = './log/pggan/128/b0/siamese_aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 4000
start_epoch = 0
train_batch = 100
test_batch = 200
lr = 0.01
schedule = [500, 1000, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/siamese_aug' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = []

# augmentation
blur_prob = 0.9
blog_sig = 0.5
jpg_prob = 0.9

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [9]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

# Dataset

In [10]:
train_dir = os.path.join(source_dir, '100_shot')
# source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch, pin_memory=True)
val_target_ = SiameseNetworkDataset(datasets.ImageFolder(val_target_dir), transform=val_aug, should_invert=False)
val_target_loader = DataLoader(val_target_, shuffle=True, num_workers=num_workers, batch_size=test_batch, pin_memory=True)
val_source_ = SiameseNetworkDataset(datasets.ImageFolder(val_source_dir), transform=val_aug, should_invert=False)
val_source_loader = DataLoader(val_source_, batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [11]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.0})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/siamese_aug/checkpoint.pth.tar'


In [12]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 4.17M


In [13]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [14]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [15]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += torch.norm(param, p=1)
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        sp_loss += torch.norm(param - teacher_model_weights[name], p=1)
    return sp_loss

# Loss

In [17]:
criterion = ContrastiveLoss(margin=4.0).cuda()
# optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [18]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

# Train

In [19]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
        
        outputs0 = student_model(inputs0)
        outputs1 = student_model(inputs1)
        
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)
        
        loss_main = criterion(outputs0, outputs1, targets)
        loss_cls = 0
        loss_sp = 0
#         loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())
        losses.update(loss.data.tolist(), inputs0.size(0))
        main_losses.update(loss_main.tolist(), inputs0.size(0))
        arc.update(auroc, inputs0.size(0))
        cls_losses.update(loss_cls, inputs0.size(0))
        sp_losses.update(loss_sp, inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

#         if batch_idx % 10 == 0:
    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{total:.4f} | SPLoss:{eta:.4f} | | CLLoss:{sp:.4f} | top1:{tp1:.4f} | AUROC:{arc:.4f}'.format(
                 batch=batch_idx+1, size=len(train_loader),loss=losses.avg, total=main_losses.avg, eta=sp_losses.avg, sp=cls_losses.avg, tp1=top1.avg, arc=auroc))
    return (losses.avg, top1.avg, arc.avg)

In [20]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            loss_main = criterion(outputs0, outputs1, targets)
            loss_cls = 0
            loss_sp = 0
#             loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())

            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))
            main_losses.update(loss_main.tolist(), inputs0.size(0))
            arc.update(auroc, inputs0.size(0))
            cls_losses.update(loss_cls, inputs0.size(0))
            sp_losses.update(loss_sp, inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('{batch}/{size} | Loss:{loss:} | MainLoss:{total:.4f} | SPLoss:{eta:.4f} | CLLoss:{sp:.4f} | top1:{tp1:.4f} | AUROC:{arc:.4f}'.format(
         batch=batch_idx+1, size=len(val_loader), total=main_losses.avg, eta=sp_losses.avg, sp=cls_losses.avg, loss=losses.avg, tp1=top1.avg, arc=auroc))
    return (losses.avg, top1.avg, arc.avg)

In [21]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [22]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
#     if epoch in iter_time:
#         print("iterative training: feedback {}".format(epoch))
#         teacher_model.load_state_dict(student_model.state_dict())
#         teacher_model_weights = {}
#         for name, param in teacher_model.named_parameters():
#             teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc,test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, source_loss, train_acc, test_acc, source_acc, train_auroc, test_auroc, source_auroc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.001000
2/2 | Loss:212.2732 | MainLoss:5.6436 | SPLoss:2066.2957 | | CLLoss:0.0000 | top1:50.0000 | AUROC:0.6877
39/39 | Loss:118.35365940974309 | MainLoss:7.7051 | SPLoss:1106.4852 | CLLoss:0.0000 | top1:49.3590 | AUROC:0.5142
161/161 | Loss:114.97431213759188 | MainLoss:4.3258 | SPLoss:1106.4872 | CLLoss:0.0000 | top1:49.7196 | AUROC:0.9976

Epoch: [2 | 4000] LR: 0.001060
2/2 | Loss:194.0975 | MainLoss:5.2053 | SPLoss:1888.9221 | | CLLoss:0.0000 | top1:54.5000 | AUROC:0.8111

Epoch: [3 | 4000] LR: 0.001120
2/2 | Loss:352.3655 | MainLoss:5.9318 | SPLoss:3464.3372 | | CLLoss:0.0000 | top1:54.0000 | AUROC:0.8846

Epoch: [4 | 4000] LR: 0.001180
2/2 | Loss:120.1008 | MainLoss:5.3066 | SPLoss:1147.9410 | | CLLoss:0.0000 | top1:46.0000 | AUROC:0.9867

Epoch: [5 | 4000] LR: 0.001240
2/2 | Loss:138.5476 | MainLoss:5.4140 | SPLoss:1331.3357 | | CLLoss:0.0000 | top1:47.0000 | AUROC:0.8368

Epoch: [6 | 4000] LR: 0.001300
2/2 | Loss:96.7475 | MainLoss:4.8913 | SPLoss:918.5

2/2 | Loss:280.9012 | MainLoss:0.7207 | SPLoss:2801.8057 | | CLLoss:0.0000 | top1:56.0000 | AUROC:0.9893

Epoch: [58 | 4000] LR: 0.004000
2/2 | Loss:190.6615 | MainLoss:1.1851 | SPLoss:1894.7643 | | CLLoss:0.0000 | top1:50.0000 | AUROC:0.9763

Epoch: [59 | 4000] LR: 0.004000
2/2 | Loss:263.8791 | MainLoss:0.8221 | SPLoss:2630.5698 | | CLLoss:0.0000 | top1:43.0000 | AUROC:0.9874

Epoch: [60 | 4000] LR: 0.004000
2/2 | Loss:191.1903 | MainLoss:1.1647 | SPLoss:1900.2559 | | CLLoss:0.0000 | top1:47.5000 | AUROC:0.9944

Epoch: [61 | 4000] LR: 0.004000
2/2 | Loss:275.7191 | MainLoss:0.8943 | SPLoss:2748.2480 | | CLLoss:0.0000 | top1:49.0000 | AUROC:0.9714

Epoch: [62 | 4000] LR: 0.004000
2/2 | Loss:190.3925 | MainLoss:0.8531 | SPLoss:1895.3943 | | CLLoss:0.0000 | top1:46.0000 | AUROC:0.9808

Epoch: [63 | 4000] LR: 0.004000
2/2 | Loss:261.6803 | MainLoss:0.8410 | SPLoss:2608.3923 | | CLLoss:0.0000 | top1:40.5000 | AUROC:0.9690

Epoch: [64 | 4000] LR: 0.004000
2/2 | Loss:190.5108 | MainLoss:1.2

2/2 | Loss:243.0679 | MainLoss:0.8012 | SPLoss:2422.6663 | | CLLoss:0.0000 | top1:55.5000 | AUROC:0.9604

Epoch: [116 | 4000] LR: 0.003997
2/2 | Loss:191.3262 | MainLoss:0.9212 | SPLoss:1904.0497 | | CLLoss:0.0000 | top1:46.0000 | AUROC:0.9639

Epoch: [117 | 4000] LR: 0.003997
2/2 | Loss:252.1179 | MainLoss:0.8282 | SPLoss:2512.8972 | | CLLoss:0.0000 | top1:54.0000 | AUROC:0.9741

Epoch: [118 | 4000] LR: 0.003997
2/2 | Loss:190.1585 | MainLoss:1.2425 | SPLoss:1889.1592 | | CLLoss:0.0000 | top1:46.0000 | AUROC:0.9442

Epoch: [119 | 4000] LR: 0.003997
2/2 | Loss:241.1336 | MainLoss:0.8341 | SPLoss:2402.9944 | | CLLoss:0.0000 | top1:52.5000 | AUROC:0.9928

Epoch: [120 | 4000] LR: 0.003997
2/2 | Loss:190.4936 | MainLoss:0.5035 | SPLoss:1899.9012 | | CLLoss:0.0000 | top1:50.5000 | AUROC:0.9960

Epoch: [121 | 4000] LR: 0.003997
2/2 | Loss:248.3704 | MainLoss:0.7891 | SPLoss:2475.8125 | | CLLoss:0.0000 | top1:50.5000 | AUROC:0.9896

Epoch: [122 | 4000] LR: 0.003997
2/2 | Loss:189.7964 | MainL

2/2 | Loss:239.0923 | MainLoss:0.6643 | SPLoss:2384.2800 | | CLLoss:0.0000 | top1:48.0000 | AUROC:0.9825

Epoch: [174 | 4000] LR: 0.003991
2/2 | Loss:191.4793 | MainLoss:0.7016 | SPLoss:1907.7775 | | CLLoss:0.0000 | top1:48.5000 | AUROC:0.9844

Epoch: [175 | 4000] LR: 0.003991
2/2 | Loss:231.7373 | MainLoss:0.8949 | SPLoss:2308.4236 | | CLLoss:0.0000 | top1:50.0000 | AUROC:0.9944

Epoch: [176 | 4000] LR: 0.003991
2/2 | Loss:191.0433 | MainLoss:0.8147 | SPLoss:1902.2855 | | CLLoss:0.0000 | top1:48.5000 | AUROC:0.9839

Epoch: [177 | 4000] LR: 0.003990
2/2 | Loss:238.6645 | MainLoss:1.0654 | SPLoss:2375.9912 | | CLLoss:0.0000 | top1:48.0000 | AUROC:0.9820

Epoch: [178 | 4000] LR: 0.003990
2/2 | Loss:189.6627 | MainLoss:0.9706 | SPLoss:1886.9208 | | CLLoss:0.0000 | top1:55.5000 | AUROC:0.9610

Epoch: [179 | 4000] LR: 0.003990
2/2 | Loss:230.1877 | MainLoss:0.9875 | SPLoss:2292.0024 | | CLLoss:0.0000 | top1:57.5000 | AUROC:0.9791

Epoch: [180 | 4000] LR: 0.003990
2/2 | Loss:190.4658 | MainL

2/2 | Loss:222.6140 | MainLoss:0.8473 | SPLoss:2217.6672 | | CLLoss:0.0000 | top1:49.0000 | AUROC:0.9876

Epoch: [232 | 4000] LR: 0.003980
2/2 | Loss:186.5872 | MainLoss:0.9825 | SPLoss:1856.0465 | | CLLoss:0.0000 | top1:54.5000 | AUROC:0.9650

Epoch: [233 | 4000] LR: 0.003980
2/2 | Loss:227.1333 | MainLoss:0.7995 | SPLoss:2263.3381 | | CLLoss:0.0000 | top1:53.0000 | AUROC:0.9712

Epoch: [234 | 4000] LR: 0.003980
2/2 | Loss:188.1563 | MainLoss:0.9921 | SPLoss:1871.6421 | | CLLoss:0.0000 | top1:56.5000 | AUROC:0.9743

Epoch: [235 | 4000] LR: 0.003979
2/2 | Loss:221.4995 | MainLoss:1.3215 | SPLoss:2201.7803 | | CLLoss:0.0000 | top1:45.5000 | AUROC:0.9485

Epoch: [236 | 4000] LR: 0.003979
2/2 | Loss:187.4090 | MainLoss:0.7707 | SPLoss:1866.3826 | | CLLoss:0.0000 | top1:51.0000 | AUROC:0.9932

Epoch: [237 | 4000] LR: 0.003979
2/2 | Loss:228.7746 | MainLoss:1.4241 | SPLoss:2273.5046 | | CLLoss:0.0000 | top1:52.5000 | AUROC:0.9732

Epoch: [238 | 4000] LR: 0.003979
2/2 | Loss:188.3194 | MainL

...

2/2 | Loss:1.1748 | MainLoss:0.5909 | SPLoss:5.8394 | | CLLoss:0.0000 | top1:49.0000 | AUROC:0.9936

Epoch: [3713 | 4000] LR: 0.000000
2/2 | Loss:1.1196 | MainLoss:0.5356 | SPLoss:5.8394 | | CLLoss:0.0000 | top1:54.0000 | AUROC:0.9910

Epoch: [3714 | 4000] LR: 0.000000
2/2 | Loss:1.3582 | MainLoss:0.7743 | SPLoss:5.8394 | | CLLoss:0.0000 | top1:52.0000 | AUROC:0.9852

Epoch: [3715 | 4000] LR: 0.000000
2/2 | Loss:1.2523 | MainLoss:0.6684 | SPLoss:5.8393 | | CLLoss:0.0000 | top1:46.0000 | AUROC:0.9904

Epoch: [3716 | 4000] LR: 0.000000
2/2 | Loss:1.2717 | MainLoss:0.6878 | SPLoss:5.8393 | | CLLoss:0.0000 | top1:50.5000 | AUROC:0.9756

Epoch: [3717 | 4000] LR: 0.000000
2/2 | Loss:1.0017 | MainLoss:0.4178 | SPLoss:5.8393 | | CLLoss:0.0000 | top1:50.5000 | AUROC:0.9980

Epoch: [3718 | 4000] LR: 0.000000
2/2 | Loss:1.2704 | MainLoss:0.6864 | SPLoss:5.8393 | | CLLoss:0.0000 | top1:45.0000 | AUROC:0.9828

Epoch: [3719 | 4000] LR: 0.000000
2/2 | Loss:0.9310 | MainLoss:0.3471 | SPLoss:5.8393 | |

2/2 | Loss:1.0507 | MainLoss:0.4669 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:53.0000 | AUROC:0.9976

Epoch: [3773 | 4000] LR: 0.000000
2/2 | Loss:1.0784 | MainLoss:0.4946 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:44.0000 | AUROC:0.9968

Epoch: [3774 | 4000] LR: 0.000000
2/2 | Loss:0.9641 | MainLoss:0.3803 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:60.5000 | AUROC:0.9929

Epoch: [3775 | 4000] LR: 0.000000
2/2 | Loss:1.2960 | MainLoss:0.7122 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:51.5000 | AUROC:0.9936

Epoch: [3776 | 4000] LR: 0.000000
2/2 | Loss:1.0329 | MainLoss:0.4491 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:48.5000 | AUROC:0.9940

Epoch: [3777 | 4000] LR: 0.000000
2/2 | Loss:1.0652 | MainLoss:0.4813 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:47.5000 | AUROC:0.9940

Epoch: [3778 | 4000] LR: 0.000000
2/2 | Loss:1.1093 | MainLoss:0.5255 | SPLoss:5.8383 | | CLLoss:0.0000 | top1:48.5000 | AUROC:0.9844

Epoch: [3779 | 4000] LR: 0.000000
2/2 | Loss:1.0901 | MainLoss:0.5062 | SPLoss:5.8382 | |

2/2 | Loss:1.4367 | MainLoss:0.8529 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:48.0000 | AUROC:0.9808

Epoch: [3833 | 4000] LR: 0.000000
2/2 | Loss:1.0344 | MainLoss:0.4506 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:51.5000 | AUROC:0.9928

Epoch: [3834 | 4000] LR: 0.000000
2/2 | Loss:1.1038 | MainLoss:0.5200 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:52.0000 | AUROC:0.9860

Epoch: [3835 | 4000] LR: 0.000000
2/2 | Loss:1.2390 | MainLoss:0.6553 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:51.0000 | AUROC:0.9880

Epoch: [3836 | 4000] LR: 0.000000
2/2 | Loss:1.2587 | MainLoss:0.6749 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:51.5000 | AUROC:0.9972

Epoch: [3837 | 4000] LR: 0.000000
2/2 | Loss:1.0402 | MainLoss:0.4565 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:43.5000 | AUROC:0.9949

Epoch: [3838 | 4000] LR: 0.000000
2/2 | Loss:1.0531 | MainLoss:0.4694 | SPLoss:5.8376 | | CLLoss:0.0000 | top1:47.0000 | AUROC:0.9828

Epoch: [3839 | 4000] LR: 0.000000
2/2 | Loss:1.3333 | MainLoss:0.7495 | SPLoss:5.8375 | |

2/2 | Loss:1.4968 | MainLoss:0.9131 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:53.0000 | AUROC:0.9943

Epoch: [3893 | 4000] LR: 0.000000
2/2 | Loss:1.1834 | MainLoss:0.5997 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:49.5000 | AUROC:0.9968

Epoch: [3894 | 4000] LR: 0.000000
2/2 | Loss:0.9909 | MainLoss:0.4072 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:56.5000 | AUROC:0.9971

Epoch: [3895 | 4000] LR: 0.000000
2/2 | Loss:1.2469 | MainLoss:0.6632 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:47.0000 | AUROC:0.9872

Epoch: [3896 | 4000] LR: 0.000000
2/2 | Loss:1.0955 | MainLoss:0.5118 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:49.0000 | AUROC:0.9932

Epoch: [3897 | 4000] LR: 0.000000
2/2 | Loss:1.1882 | MainLoss:0.6045 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:54.5000 | AUROC:0.9924

Epoch: [3898 | 4000] LR: 0.000000
2/2 | Loss:1.3235 | MainLoss:0.7398 | SPLoss:5.8371 | | CLLoss:0.0000 | top1:46.5000 | AUROC:0.9862

Epoch: [3899 | 4000] LR: 0.000000
2/2 | Loss:1.3716 | MainLoss:0.7879 | SPLoss:5.8371 | |

39/39 | Loss:7.6945169277680225 | MainLoss:7.1108 | SPLoss:5.8369 | CLLoss:0.0000 | top1:50.0000 | AUROC:0.4255
161/161 | Loss:0.9868831393131957 | MainLoss:0.4032 | SPLoss:5.8369 | CLLoss:0.0000 | top1:50.2804 | AUROC:0.9960

Epoch: [3952 | 4000] LR: 0.000000
2/2 | Loss:1.3116 | MainLoss:0.7279 | SPLoss:5.8369 | | CLLoss:0.0000 | top1:46.0000 | AUROC:0.9872

Epoch: [3953 | 4000] LR: 0.000000
2/2 | Loss:1.5317 | MainLoss:0.9480 | SPLoss:5.8369 | | CLLoss:0.0000 | top1:47.0000 | AUROC:0.9828

Epoch: [3954 | 4000] LR: 0.000000
2/2 | Loss:1.0550 | MainLoss:0.4713 | SPLoss:5.8369 | | CLLoss:0.0000 | top1:48.5000 | AUROC:0.9992

Epoch: [3955 | 4000] LR: 0.000000
2/2 | Loss:1.0948 | MainLoss:0.5111 | SPLoss:5.8369 | | CLLoss:0.0000 | top1:46.5000 | AUROC:0.9891

Epoch: [3956 | 4000] LR: 0.000000
2/2 | Loss:0.9222 | MainLoss:0.3385 | SPLoss:5.8369 | | CLLoss:0.0000 | top1:47.5000 | AUROC:0.9984

Epoch: [3957 | 4000] LR: 0.000000
2/2 | Loss:1.3716 | MainLoss:0.7879 | SPLoss:5.8369 | | CLLoss:0